# Ноутбук для расчета MDE до запуска эксперимента

Этот ноутбук нужен для того, чтобы до запуска теста рассчитывать приблизительное необходимое количество наблюдений на эксперимент с учетом ожидаемогого размера эффекта

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import logging
import sys
import re
from citymobil_python_mysql_wrapper import MysqlWrapper
import pyexasol
from random import randrange
from itertools import combinations
from datetime import datetime, timedelta
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 999)

In [3]:
# импорт модулей с sql-запросами и функциями для обработки
import utils as u
import query as q

# Запрашиваем из БД все необходимые данные

In [4]:
cred = pd.read_json(r'/Users/skostuchik/crd_exa.json')
user = cred.iloc[0, 0]
password = cred.iloc[0, 1]

C = pyexasol.connect(dsn='ex1..3.city-srv.ru:8563', user=user, password=password, fetch_dict=True)

## !!!В следующей ячейке необимо ввести значения переменных!!!

In [5]:
# в lift_dict необходимо указать ожидаемый размер лифта для интересующих метрик
lift_dict = {'AR': 0.02,
 'RR': 0.02,
 'FR': 0.02,
 'CHURN_7DAY': -0.02,
 'CHURN_14DAY': -0.02,
 'CHURN_21DAY': -0.02,
 'MPH_GROSS': 0.02,
 'MPH_NET': 0.02,
 'ORGANIC_MPH_GROSS': 0.02,
 'ORGANIC_MPH_NET': 0.02,
 'SHpD': 0.01,
 'DE': 0.02,
 'TPD': 0.02,
 'OF2R': 0.02,
 'GMVPT': 0.02,
 'COPT': -0.02,
 'DIPT': -0.02,
 'COMPT': 0.02,
 'DIST_MEAN': 0.02,
 'DIST_MEDIAN': 0.02}

# необходимо указать locality_id
# по умолчанию данные берутся за последние 2 недели (принеобходимости можно поставить другой период)
params = {'locality_id': 338,
          'date_from' : (datetime.today() - timedelta(days=14)).strftime("%Y-%m-%d"),
          'date_to' : (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")}

# необходимо указать запрос, который возвращает id водителей и сплит
drivers_cte = '''
    select d.ID
                  , case when mod(UDF.CRC32(CONCAT(d.ID, '_20210406')), 100) < 50 then 'B' else 'A' end as exp_group
        from REPLICA.DRIVERS d
                      left join md.LOCALITY l on l.LOCALITY_RK = d.ID_LOCALITY
        where 1 = 1
               and LOCALITY_RK in ({locality_id})
               and d.role = 11
               and d.is_test != 1
               and d.status = 'A'
               and LAST_ORDER_DATE >= current_date - 30
'''

Важно:
- В запросе должны быть поля ID (id водителя) и exp_group (сплит)
- Запрос далее подставляется в другие запросы в виде CTE, а значит в нем нельзя использовать CTE - вместо этого следует использовать подзапрос (вместо with t as (...) нужно select * from (select ...))

In [6]:
query_params = {'locality_id':params.get('locality_id'),
                'drivers_cte':drivers_cte.format(locality_id=params.get('locality_id')),
                'date_from':params.get('date_from'), 'date_to':params.get('date_to')}

In [7]:
e_ar, e_of2r, e_mph, e_copt, e_dist, e_ch = C.execute(q.ar_query.format(**query_params)),\
C.execute(q.of2r_query.format(**query_params)), C.execute(q.mph_query.format(**query_params)),\
C.execute(q.copt_query.format(**query_params)), C.execute(q.dist_query.format(**query_params)),\
C.execute(q.churn_query.format(**query_params))

In [8]:
ar_suggest_data, of2r_data, mph_data, copt_data, dist_data, churn_data = \
pd.DataFrame(e_ar.fetchall()), pd.DataFrame(e_of2r.fetchall()),\
pd.DataFrame(e_mph.fetchall()), pd.DataFrame(e_copt.fetchall()),\
pd.DataFrame(e_dist.fetchall()), pd.DataFrame(e_ch.fetchall())

In [9]:
#есть косяк с наполнением таблицы SH в DWH
mph_data = mph_data[mph_data['SUPPLY_HOURS'].notna()]

In [10]:
# словарь с конфигом - тут все метрики и то, как их надо считать
config = {
    'alpha':0.01, 'power':0.9,
    'metrics': {
        'AR':{
            'df':ar_suggest_data,
            'add_breakdown':{
                'Total':'Total',
                'ChainOffer':'SPECIFICATION_NAME',
                '0 - 3 km':'DISTANCE_CATEGORY','3 - 6 km':'DISTANCE_CATEGORY',
                '6 - 10 km':'DISTANCE_CATEGORY','+10 km':'DISTANCE_CATEGORY'
            },
            'options':{
                'x':['ACCEPT'],
                'n':['FSS_ID', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'RR':{
            'df':ar_suggest_data,
            'add_breakdown':{
                'Total':'Total',
                'ChainOffer':'SPECIFICATION_NAME',
                '0 - 3 km':'DISTANCE_CATEGORY','3 - 6 km':'DISTANCE_CATEGORY',
                '6 - 10 km':'DISTANCE_CATEGORY','+10 km':'DISTANCE_CATEGORY'
            },
            'options':{
                'x':['REJECT'],
                'n':['FSS_ID', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'FR':{
            'df':ar_suggest_data,
            'add_breakdown':{
                'Total':'Total',
                'ChainOffer':'SPECIFICATION_NAME',
                '0 - 3 km':'DISTANCE_CATEGORY','3 - 6 km':'DISTANCE_CATEGORY',
                '6 - 10 km':'DISTANCE_CATEGORY','+10 km':'DISTANCE_CATEGORY'
            },
            'options':{
                'x':['FRAUD'],
                'n':['FSS_ID', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'CHURN_7DAY':{
            'df':churn_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['CHURN_7DAY'],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'CHURN_14DAY':{
            'df':churn_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['CHURN_14DAY'],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'CHURN_21DAY':{
            'df':churn_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['CHURN_21DAY'],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'binomial'
        },
        'MPH_GROSS':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['MONEY_GROSS', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'MPH_NET':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['MONEY_NET', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'ORGANIC_MPH_GROSS':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['ORGANIC_MONEY_GROSS', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'ORGANIC_MPH_NET':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['ORGANIC_MONEY_NET', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'SHpD':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['SUPPLY_HOURS', np.sum],
                #'y':['DRIVER_RK', pd.Series.nunique],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            #'type':'ratio'
            'type':'continious'
        },
        'DE':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total', 'Not_park':'PARK', 'Park':'PARK', 'not_brand':'BRAND', 'brand':'BRAND',
                'TOP':'SEGMENT', 'HIGH':'SEGMENT'
            },
            'options':{
                'x':['ON_TRIP', np.sum],
                'y':['SUPPLY_HOURS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'TPD':{
            'df':mph_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['TRIPS', np.sum],
                #'y':['DRIVER_RK', pd.Series.nunique],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            #'type':'ratio'
            'type':'continious'
        },
        'OF2R':{
            'df':of2r_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['RIDES', np.sum],
                'y':['OFFERS', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'GMVPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['GMV', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'COPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['CONTRIBUTION', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'DIPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['DI', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'COMPT':{
            'df':copt_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['COMMISSION', np.sum],
                'y':['RIDES', np.sum],
                'n':['DRIVER_RK', pd.Series.nunique]
            },
            'type':'ratio'
        },
        'DIST_MEAN':{
            'df':dist_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['EXP_DIST_KM', np.mean],
                'n':['ORDER_RK', pd.Series.nunique]
            },
            'type':'continious'
        },
        'DIST_MEDIAN':{
            'df':dist_data,
            'add_breakdown':{
                'Total':'Total'
            },
            'options':{
                'x':['EXP_DIST_KM', np.median],
                'n':['ORDER_RK', pd.Series.nunique]
            },
            'type':'continious'
        }
    }
}

In [11]:
# непосредственно расчет значений метрик
result_dict = {}

for metric,v in tqdm(config.get('metrics').items()):
    lift=lift_dict[metric]
    try:
        src = u.mde_calculation(metric=metric, m_type=v.get('type'), df=v.get('df'),
                                     lift=lift, cfg=config)
        result_dict.update({(metric,lift):src})
    except:
        continue

100%|██████████| 20/20 [00:00<00:00, 54.65it/s]


In [12]:
mde_data = pd.DataFrame(result_dict, index=['mde_per_sample']).T

In [13]:
mde_data

,,mde_per_sample
AR,0.02,9050
RR,0.02,8750
FR,0.02,1409
CHURN_7DAY,-0.02,4545
CHURN_14DAY,-0.02,9118
CHURN_21DAY,-0.02,6827
MPH_GROSS,0.02,172944183307
MPH_NET,0.02,172679165553
ORGANIC_MPH_GROSS,0.02,204825781597
ORGANIC_MPH_NET,0.02,213199824572


MDE рассчитывается для 1 сплита. В config для каждой метрики можно посмотреть юнит (options.n) - например для AR это саджаст, для churn - водитель

In [ ]:
#экспорт в эксель
mde_data.to_excel(r'mde_data_initial.xlsx', sheet_name='mde_data_initial', index = True)